In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from keras.preprocessing.text import Tokenizer
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences

In [2]:
os.listdir('/Users/reza/Desktop/PhD_thesis/datasets/flickr30k_images')


['.DS_Store', 'flickr30k_images', 'results.csv']

In [3]:
metadata = pd.read_csv('/Users/reza/Desktop/PhD_thesis/datasets/flickr30k_images/results.csv',delimiter='|',engine='python')
metadata = metadata.dropna()
is_NaN = metadata.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = metadata[row_has_NaN]
print(rows_with_NaN)
metadata.head()

Empty DataFrame
Columns: [image_name,  comment_number,  comment]
Index: []


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .


In [4]:
#from tensorflow.keras.utils import load_img

In [5]:
def load_image(name):
    img=image.load_img(name,target_size=(32,32,3))
    img=image.img_to_array(img)
    img=np.reshape(img,(32*32*3))
    return img

In [6]:
print(len(metadata['image_name'].unique()))
print(len(metadata['image_name']))

31783
158914


In [7]:
image_arr = []
sentence_arr = []
for ind in range(5000):
    if ind % 5 != 0:
        continue
    image_location = (metadata.iloc[ind,:]['image_name'])
    sentence = (metadata.iloc[ind,:][' comment'])
    
    
    image_arr.append(load_image('/Users/reza/Desktop/PhD_thesis/datasets/flickr30k_images/flickr30k_images/flickr30k_images/'+str(image_location)) )
    sentence_arr.append('<SOS>'+sentence+'<EOS>')
    
        
Images =  np.array(image_arr)


In [8]:
image_arr=load_image('/Users/reza/Desktop/PhD_thesis/datasets/flickr30k_images/flickr30k_images/36979.jpg')

In [9]:
image_arr.shape

(3072,)

In [10]:
Images.shape

(1000, 3072)

In [11]:
len(sentence_arr)

1000

In [12]:
sentence_arr[0]

'<SOS> Two young guys with shaggy hair look at their hands while hanging out in the yard .<EOS>'

In [13]:
sentence_arr[1]

'<SOS> Several men in hard hats are operating a giant pulley system .<EOS>'

In [14]:
sentence_arr[2]

'<SOS> A child in a pink dress is climbing up a set of stairs in an entry way .<EOS>'

In [15]:
metadata.iloc[0:15,:]


,image_name,comment_number,comment
0,1000092795.jpg,0,Two young guys with shaggy hair look at their...
1,1000092795.jpg,1,"Two young , White males are outside near many..."
2,1000092795.jpg,2,Two men in green shirts are standing in a yard .
3,1000092795.jpg,3,A man in a blue shirt standing in a garden .
4,1000092795.jpg,4,Two friends enjoy time spent together .
5,10002456.jpg,0,Several men in hard hats are operating a gian...
6,10002456.jpg,1,Workers look down from up above on a piece of...
7,10002456.jpg,2,Two men working on a machine wearing hard hats .
8,10002456.jpg,3,Four men on top of a tall structure .
9,10002456.jpg,4,Three men on a large rig .


In [16]:
def tokenize(x):
    tokenizer=Tokenizer()
    tokenizer.fit_on_texts(x)
    t=tokenizer.texts_to_sequences(x)
    return t, tokenizer

In [17]:
text_tokenized,tokenizer=tokenize(['ehasan is reza brother','reza is student'])
print('text_tokenized is :',text_tokenized)
print('tokenizer is :',tokenizer)

text_tokenized is : [[3, 1, 2, 4], [2, 1, 5]]
tokenizer is : <keras.preprocessing.text.Tokenizer object at 0x12e6e6b20>


In [19]:
def pad(x, length=None):
    padding=pad_sequences(x,padding='post',maxlen=length)
    return padding

In [21]:
def preprocess(sentences):
    text_tokenized, text_tokenizer= tokenize(sentences)
    text_pad=pad(text_tokenized)
    return text_pad , text_tokenizer
Sentence, token_Sentence=preprocess(sentence_arr)

In [22]:
print("Sentence vocabulary size:", len(token_Sentence.word_index))
print("Sentence Longest sentence size:", len(Sentence[0]))

Sentence vocabulary size: 2302
Sentence Longest sentence size: 51


In [24]:
type(token_Sentence)

keras.preprocessing.text.Tokenizer

In [26]:
Images.shape , Sentence.shape

((1000, 3072), (1000, 51))

In [ ]:
#added this to see what is difference
#ezafe shod

: 